#### Imports

In [1]:
import pandas as pd
import numpy as np
import folium
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
import random as rd
import sklearn.model_selection as sk
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.impute import SimpleImputer
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


#### Load Data Files

In [2]:
# test files
test_month_1 = pd.read_csv('data/test_month_1.csv')
test_month_2 = pd.read_csv('data/test_month_2.csv')
test_month_3 = pd.read_csv('data/test_month_3.csv')

# train files
train_month_1 = pd.read_csv('data/train_month_1.csv')
train_month_2 = pd.read_csv('data/train_month_2.csv')
train_month_3_with_target = pd.read_csv('data/train_month_3_with_target.csv')

#### Transform

In [3]:
# convert the dates into days before 2018 
train_month_3_with_target["customer_since_all"] = (pd.to_datetime("2018-01") - pd.to_datetime(train_month_3_with_target["customer_since_all"])).dt.days / 365.5
train_month_3_with_target["customer_birth_date"] = (pd.to_datetime("2018-01") - pd.to_datetime(train_month_3_with_target["customer_birth_date"])).dt.days / 365.5
train_month_3_with_target["customer_since_bank"] = (pd.to_datetime("2018-01") - pd.to_datetime(train_month_3_with_target["customer_since_bank"])).dt.days / 365.5

# dummies for categorical variables
train_month_3_with_target = pd.DataFrame(pd.get_dummies(train_month_3_with_target, 
                                   columns = ["customer_education", "customer_occupation_code", "customer_children", "customer_relationship"], 
                                   dummy_na=True))

# drop unwanted columns client id 
train_month_3_with_target = train_month_3_with_target.drop(["client_id"], axis=1)

# retain information of NAs in customer_since_all_na and customer_since_bank_na
train_month_3_with_target['customer_since_all_na'] = np.where(train_month_3_with_target.customer_since_all.notnull(), '0', '1')
train_month_3_with_target['customer_since_bank_na'] = np.where(train_month_3_with_target.customer_since_bank.notnull(), '0', '1')

#### Split the Set (rework this maybe)

In [62]:
# Train Test Split to prevent data pollution
y = train_month_3_with_target['target']
X = train_month_3_with_target.drop(['target'], axis = 1)
X_train, X_val, y_train, y_val = sk.train_test_split(X, y, test_size=0.3, stratify=y)

#### Imputing and Feature Engineering

In [63]:
# mean impute for rest of missings
imputer = SimpleImputer(missing_values=np.nan, strategy='mean')

imputer = imputer.fit(X_train)
X_train = pd.DataFrame(imputer.transform(X_train), columns= X_train.columns)

X_val = pd.DataFrame(imputer.transform(X_val), columns= X_val.columns)

# calculate the frequency of churners in the training set an map the values to the validation set
X_train.reset_index(drop=True, inplace=True)
y_train.reset_index(drop=True, inplace=True)



# aggr_mean = aggr["target"].mean()
#train_month_3_with_target = train_month_3_with_target.drop(["customer_postal_code"], axis=1)

In [67]:
aggr = pd.concat([X_train, y_train], axis = 1).groupby("customer_postal_code")[["customer_postal_code", "target"]].sum()
aggr["customer_postal_code"] == aggr["customer_postal_code"].astype(int)

ag_new = pd.Series(aggr.target.values,index=aggr.customer_postal_code).to_dict()
#ag_new = aggr.set_index("customer_postal_code").T.to_dict("list")

#
X_train["customer_postal_code"] = X_train["customer_postal_code"].astype(int)
print((set(X_train["customer_postal_code"].unique()) - set(pd.concat([X_train, y_train], axis = 1)["customer_postal_code"].unique())))
X_train["customer_postal_code"].map(ag_new).isna().sum(axis = 0) #.fillna(aggr["target"].mean())

set()


44425

In [93]:
# difference after merge 
set(pd.concat([X_train, y_train], axis = 1)["customer_postal_code"].unique()) - set(X_train["customer_postal_code"].unique())

# difference after group by 
pd.concat([X_train, y_train], axis = 1).groupby("customer_postal_code")["customer_postal_code"].count()
#set(pd.concat([X_train, y_train], axis = 1)["customer_postal_code"].unique()) - set(X_train["customer_postal_code"].unique())


customer_postal_code
0         9
1000     71
1020     78
1030    180
1040     59
       ... 
9982      8
9988     10
9990    219
9991     69
9992      2
Name: customer_postal_code, Length: 1031, dtype: int64

In [90]:

# group by appears to remove certain postal codes?????? But WHY???
aggr = pd.concat([X_train, y_train], axis = 1).groupby("customer_postal_code")[["customer_postal_code", "target"]].sum().drop(["customer_postal_code"], axis=1)
len(set(pd.concat([X_train, y_train], axis = 1)["customer_postal_code"].unique()) - set(pd.concat([X_train, y_train], axis = 1).groupby("customer_postal_code")[["customer_postal_code", "target"]].sum()["customer_postal_code"].unique()))






1019

#### End of Script

In [7]:
print("==================================================================")
print("cleaning.ipynb has been successfully run!")

cleaning.ipynb has been successfully run!
